In [28]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler
import seaborn as sns
import datetime as date
from sklearn.metrics import mean_absolute_error
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [29]:
main_df =         pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
holdout_df =      pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test.csv')
mini_holdout_df = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv')

In [30]:
# main_df["Year Sold"] =         [int(main_df['date'].loc[i][:4])         for i in range(len(main_df['date']))]
# holdout_df["Year Sold"] =      [int(holdout_df['date'].loc[i][:4])      for i in range(len(holdout_df['date']))]
# mini_holdout_df["Year Sold"] = [int(mini_holdout_df['date'].loc[i][:4]) for i in range(len(mini_holdout_df['date']))]

# main_df["Month Sold"] =         [int(main_df['date'].loc[i][4:6])         for i in range(len(main_df['date']))]
# holdout_df["Month Sold"] =      [int(holdout_df['date'].loc[i][4:6])      for i in range(len(holdout_df['date']))]
# mini_holdout_df["Month Sold"] = [int(mini_holdout_df['date'].loc[i][4:6]) for i in range(len(mini_holdout_df['date']))]

# main_df["Day Sold"] =         [int(main_df['date'].loc[i][6:8])         for i in range(len(main_df['date']))]
# holdout_df["Day Sold"] =      [int(holdout_df['date'].loc[i][6:8])      for i in range(len(holdout_df['date']))]
# mini_holdout_df["Day Sold"] = [int(mini_holdout_df['date'].loc[i][6:8]) for i in range(len(mini_holdout_df['date']))]

# main_df.drop('date')
# for i in range(len(main_df['Date New'])):
#     main_df['Date New'].loc[i] = int(main_df['Date New'].loc[i][:4])

# holdout_df["Date New"] = holdout_df['date']
# for i in range(len(holdout_df['Date New'])): holdout_df['Date New'].loc[i] = int(holdout_df['Date New'].loc[i][:4])

# mini_holdout_df["Date New"] = mini_holdout_df['date']
# for i in range(len(mini_holdout_df['Date New'])): mini_holdout_df['Date New'].loc[i] = int(mini_holdout_df['Date New'].loc[i][:4])

# main_df.head()
# mini_holdout_df.head()
# main_df['Date New'].dtypes

In [31]:
main_df.head(5)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


In [32]:
# Feature Engineering 

#adds whether of not they have a basement 
main_df.loc[main_df['sqft_basement'] > 0, 'has_basement'] = 'True' 
main_df.loc[main_df['sqft_basement'] == 0, 'has_basement'] = 'False' 

#adds a column spescifying whether or not the building is multi-story or ont

main_df.loc[main_df['floors'] > 1, 'is_multi_floor'] = 'True' 
main_df.loc[main_df['floors'] == 1, 'is_multi_floor'] = 'False'


# Lat of seattle (47.604291)
#adds column for if the house is norht or south of seattle
main_df.loc[main_df['lat'] > 47.604291, 'is_north_of_seattle'] = 'True' 
main_df.loc[main_df['lat'] <= 47.604291, 'is_north_of_seattle'] = 'False'

# long of seattle (-122.328970)
#adds column for if the house is east or west of seattle
main_df.loc[main_df['long'] > -122.328970, 'east_or_west_of_seattle'] = 'True' 
main_df.loc[main_df['long'] <= -122.328970, 'east_or_west_of_seattle'] = 'False'

main_df.head(5)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,price,has_basement,is_multi_floor,is_north_of_seattle,east_or_west_of_seattle
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,98038,47.3862,-122.048,3280,4033,429900.0,True,True,False,True
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,98023,47.3035,-122.382,1310,7865,233000.0,True,False,False,False
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,98116,47.5658,-122.389,1900,5800,455000.0,False,True,False,False
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,98023,47.3187,-122.390,1790,7488,258950.0,False,False,False,False
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,98117,47.6859,-122.395,1690,5962,555000.0,True,False,True,False


In [35]:
#cyclical https://www.kaggle.com/code/avanwyk/encoding-cyclical-features-for-deep-learning
def makeItCycle(df, column, period):
  df[column+'Sin'] = np.sin(2 * np.pi * df[column]/period)
  df[column+'Cos'] = np.cos(2 * np.pi * df[column]/period)
  return df.drop(columns=[column])
def preprocess(X, X_ref):
  proc = pd.DataFrame(X,copy = True)
  datetimes = pd.to_datetime(X['date'])
  proc["dayOfWeek"] = datetimes.dt.weekday
  proc['day'] = datetimes.dt.day
  proc['month'] = datetimes.dt.month
  proc['year'] = datetimes.dt.year
  proc['date'] = (datetimes - pd.datetime(1970,1,1)).dt.total_seconds()
  proc = proc.drop(columns = ["date"])
  
#   proc = makeItCycle(proc, 'day', 31)
#   proc = makeItCycle(proc, 'month', 12)
#   proc = makeItCycle(proc, 'hr', 24)
#   proc = makeItCycle(proc, 'season', 4)
  return proc

main_df = preprocess(main_df, main_df)
holdout_df = preprocess(holdout_df, holdout_df)
mini_holdout_df = preprocess(mini_holdout_df, mini_holdout_df)


/tmp/ipykernel_55442/3942059090.py:13: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  proc['date'] = (datetimes - pd.datetime(1970,1,1)).dt.total_seconds()
/tmp/ipykernel_55442/3942059090.py:13: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  proc['date'] = (datetimes - pd.datetime(1970,1,1)).dt.total_seconds()
/tmp/ipykernel_55442/3942059090.py:13: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  proc['date'] = (datetimes - pd.datetime(1970,1,1)).dt.total_seconds()


In [36]:
# main_df = main_df.drop(['date'], axis=1)
main_df = pd.get_dummies(main_df, drop_first=True)
main_df

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_lot15,price,dayOfWeek,day,month,year,has_basement_True,is_multi_floor_True,is_north_of_seattle_True,east_or_west_of_seattle_True
0,1565930130,4,3.25,3760,4675,2.0,0,0,3,8,...,4033,429900.0,1,4,11,2014,1,1,0,1
1,3279000420,3,1.75,1460,7800,1.0,0,0,2,7,...,7865,233000.0,3,15,1,2015,1,0,0,0
2,194000575,4,1.00,1340,5800,1.5,0,2,3,7,...,5800,455000.0,1,14,10,2014,0,1,0,0
3,2115510160,3,1.75,1440,8050,1.0,0,0,3,8,...,7488,258950.0,0,8,12,2014,0,0,0,0
4,7522500005,2,1.50,1780,4750,1.0,0,0,4,7,...,5962,555000.0,4,15,8,2014,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5272200045,3,1.50,1000,6914,1.0,0,0,3,7,...,6947,378000.0,3,13,11,2014,0,0,1,1
19996,9578500790,3,2.50,3087,5002,2.0,0,0,3,8,...,5183,399950.0,1,11,11,2014,0,1,0,0
19997,7202350480,3,2.50,2120,4780,2.0,0,0,3,7,...,2650,575000.0,1,30,9,2014,0,1,1,1
19998,1723049033,1,0.75,380,15000,1.0,0,0,3,5,...,15000,245000.0,4,20,6,2014,0,0,0,1


In [ ]:
# sns.pairplot(main_df)


In [ ]:
# sns.scatterplot(main_df, x='bedrooms', y='bathrooms', hue='sqft_lot')
# sns.scatterplot(main_df, x='yr_built', y='sqft_lot')
# sns.scatterplot(main_df, x='yr_built', y='sqft_living')
# sns.scatterplot(main_df, x='yr_built', y='price', hue='view')
# sns.scatterplot(main_df, x='sqft_lot', y='price')
# sns.scatterplot(main_df, x='yr_built', y='Year Sold', hue='price')
# sns.jointplot(main_df, x='yr_built', y='Year Sold')


# sns.boxenplot(main_df, x='price')
# sns.scatterplot(main_df, x='sqft_lot', y='price', hue='yr_built')
# sns.boxplot(main_df, x='Year Sold', y='price')
# sns.violinplot(main_df, x='Year Sold', y='price')
# sns.lineplot(main_df, x='yr_built', y='price')
# sns.countplot(main_df, x='Year Sold')
# sns.histplot(main_df, x='yr_built', y='price')
# sns.countplot(main_df, x='yr_built')
# sns.barplot(main_df, x='condition', y='price')
# main_df.columns
# import cartopy.crs as crs
# import cartopy.feature as cfeature
# fig = plt.figure(figsize=(12,10))

# ax = fig.add_subplot(1,1,1, projection=crs.PlateCarree())

# ax.stock_img()
# ax.coastlines()
# ax.add_feature(cfeature.STATES)

# # ax.set_extent([-125, -120, 45.5, 49.5], crs=crs.PlateCarree())
# ax.set_extent([-123, -121, 47, 48], crs=crs.PlateCarree())

# plt.scatter(x=main_df.long, y=main_df.lat, c=main_df.price, alpha=0.8, transform=crs.PlateCarree(), label = main_df.zipcode)
# plt.show()
# outline washington
# long1lat
# hue = price


In [37]:
biggest_index = 0
biggest_val = 0
for i in range(len(main_df)):
    if main_df['sqft_lot'][i] > biggest_val:
        biggest_val = main_df['sqft_lot'][i]
        biggest_index = i

print(biggest_index, biggest_val)
print(main_df['long'][biggest_index], main_df['lat'][biggest_index], main_df['yr_built'][biggest_index], main_df['zipcode'][biggest_index], main_df['price'][biggest_index])

10003 1164794
-121.909 47.6888 1915 98014 190000.0


In [38]:
# Clean up Data of null values
for column in main_df.columns:
    main_df[column].fillna(0)


In [39]:
main_df.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price', 'dayOfWeek', 'day', 'month',
       'year', 'has_basement_True', 'is_multi_floor_True',
       'is_north_of_seattle_True', 'east_or_west_of_seattle_True'],
      dtype='object')

In [40]:
XGBoost = xgb.XGBRegressor(max_depth = 4, random_state=11, n_estimators=500)

# min_child_weight = 5,
# callbacks=None,
# colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
# early_stopping_rounds=None, enable_categorical=False,
# eval_metric=None, grow_policy='depthwise',
# importance_type=None, interaction_constraints='',
# learning_rate=0.300000012, max_cat_to_onehot=4,
# max_delta_step=0, max_leaves=0, min_child_weight=1,
# missing=nan, monotone_constraints='()'

# X = main_df[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
#        'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
#        'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
#        'sqft_living15', 'sqft_lot15', 'Date New']]

# X = main_df[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
#        'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
#        'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
#        'sqft_living15', 'Year Sold', 'Month Sold', 'Day Sold']]
X = main_df.drop(['price'], axis=1)

y = main_df['price']

# ros = RandomOverSampler(random_state=11)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# xee, yee = ros.fit_resample(X_train, y_train)

XGBoost.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=11,
             reg_alpha=0, reg_lambda=1, ...)

In [41]:
predictions = XGBoost.predict(X_test)
mean_squared_error(y_test, predictions, squared=False)

129399.25972936951

mean_absolute_error(y_test, predictions), mean_squared_error(y_test, predictions, squared=False)

In [ ]:
 

# _, X_val, _, y_val = train_test_split(X, yee, test_size=0.2, random_state=42)]]

x_holdout = holdout_df[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'Year Sold', 'Month Sold', 'Day Sold']]

predictions = XGBoost.predict(x_holdout)
y_mini = pd.DataFrame()
y_mini['price'] = predictions
y_mini.to_csv('holdout-module3-team4-predictions.csv', index=False)

In [ ]:
importances = pd.DataFrame({
    'Attribute': X_train.columns,
    'Importance': XGBoost.feature_importances_
})
for i in range(len(importances)):
    importances['Importance'].loc[i] = importances["Importance"].loc[i] * 100
importances.sort_values(by='Importance', ascending=False)